In [1]:
import os
import cv2
import shutil
import random
from pathlib import Path
import pandas as pd
from tqdm import tqdm
from scenedetect import open_video, SceneManager, ContentDetector, ThresholdDetector, AdaptiveDetector, HistogramDetector, HashDetector

import tensorflow as tf
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.utils import load_img, img_to_array

from keras.preprocessing.image import load_img

/home/antonia/Documents/utcn/master/ip/VideoDescription/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-20 11:38:10.140711: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-20 11:38:10.191627: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-20 11:38:11.963884: I tensorflow/core/util/port.cc:153] oneDNN custom operations 

In [34]:
import numpy as np
import tensorflow as tf

print("NumPy version:", numpy.__version__)
print("TensorFlow version:", tf.__version__)

NumPy version: 2.1.3
TensorFlow version: 2.20.0-dev20250516


## Input/Frame Preprocessing
<blockquote>As I've said in the previous notebook ("video_frame_extraction.ipynb"), there are several techniques to choose the frames for the next step, like uniform sampling (fixed time intervals), scene change detection techniques, and ordinary frame extraction (keep only the specified number of frames).</blockquote>
<blockquote>In this notebook, I will perform this frame selection, using these 3 methods.</blockquote>
<blockquote>As I was doing my research on scene detection techniques, I discovered the <code>scenedetect</code> API (Python library which works directly with video clips). Basically, it outputs the frames where there is a significant change based on a given threshold. The <code>ContentDetector</code> detects fast cuts using weighted average of HSV changes.</blockquote>
<blockquote>There are also other options from which you can choose to perform these scene changes detection: <code>ThresholdDetector</code>, <code>AdaptiveDetector</code>, <code>HistogramDetector</code>, <code>HashDetector</code>.</blockquote>
<blockquote>I have observed that there are some video clips which do not have significant changes between frames, and neither one of the detectors have successfully extracted significant frames. These might be a problem if there is a significant amount of videos in this situation.</blockquote>
<blockquote>For starters, I will start using the <code>AdaptiveDetector</code> and observe the results.</blockquote>
<blockquote><em>AdaptiveDetector compares the difference in content between adjacent frames similar to ContentDetector except the threshold isn’t fixed, but is a rolling average of adjacent frame changes. This can help mitigate false detections in situations such as fast camera motions.</em></blockquote>

In [20]:
def detect_scenes_per_video(video_path, method):
    video = open_video(video_path)
    scene_manager = SceneManager()

    if method == "content_detector":
        scene_manager.add_detector(ContentDetector())
    elif method == "threhsold_detector":
        scene_manager.add_detector(ThresholdDetector())
    elif method == "adaptive_detector":
        scene_manager.add_detector(AdaptiveDetector())
    elif method == "histogram_detector":
        scene_manager.add_detector(HistogramDetector())
    elif method == "hash_detector":
        scene_manager.add_detector(HashDetector())
    else:
        raise ValueError(f"Unknown method '{method}'.")

    scene_manager.detect_scenes(video)
    scenes = scene_manager.get_scene_list()

    return scenes

In [48]:
def select_frames(video_path, detector_method, selected_frames_path, required_frames, frames_metadata):
    video_id = Path(video_path).stem
    
    source_dir = Path(f"data/Frames/{video_id}")
    dest_dir = Path(f"{selected_frames_path}/{video_id}")
    dest_dir.mkdir(parents=True, exist_ok=True)
    
    scenes = detect_scenes_per_video(video_path, detector_method)

    key_frame_indices = []
    for start, end in scenes:
        start_frame = start.get_frames()
        key_frame_indices.append(start_frame)
    
    selected_frames = []
    for frame_idx in key_frame_indices:
        frame_filename = f"frame{frame_idx}.jpg"
        source_frame_path = source_dir / frame_filename
        dest_frame_path = dest_dir / frame_filename
        if source_frame_path.exists():
            shutil.copy(source_frame_path, dest_frame_path)
            selected_frames.append(dest_frame_path)

    all_frames = sorted(source_dir.glob("frame*.jpg"))
    all_frame_indices = [int(f.stem.replace("frame", "")) for f in all_frames]
    
    remaining_indices = list(set(all_frame_indices) - set(key_frame_indices))
    
    extra_needed = required_frames - len(selected_frames)
    if extra_needed > 0:
        if len(remaining_indices) >= extra_needed:
            sampled = random.sample(remaining_indices, extra_needed)
        else:
            sampled = random.choices(remaining_indices, k=extra_needed)
        
        for idx in sampled:
            frame_filename = f"frame{idx}.jpg"
            source_frame_path = source_dir / frame_filename
            dest_frame_path = dest_dir / frame_filename
            if source_frame_path.exists():
                shutil.copy(source_frame_path, dest_frame_path)
                selected_frames.append(dest_frame_path)

    frames_metadata.append({
        "VideoID": video_id,
        "Key_Frames": len(set(key_frame_indices)),
        "Total_Frames": len(selected_frames)
    })

In [64]:
video_dir = Path("data/YouTubeClips")
all_videos = sorted(video_dir.glob("*.avi"))
total_videos = len(all_videos)

frames_metadata = []
failed_frames = []

with tqdm(total=total_videos, desc=f"Processing {total_videos} YouTube clips", unit="clip") as pbar:
    for video_file in all_videos:
        try:
            select_key_frames(
                video_path=str(video_file),
                detector_method="adaptive_detector",
                selected_frames_path="data/SelectedFrames",
                required_frames=50,
                frames_metadata=frames_metadata
            )
        except Exception as e:
            failed_frames.append(video_file)
        pbar.update(1)

Processing 1970 YouTube clips: 100%|██████████████| 1970/1970 [12:01<00:00,  2.73clip/s]


In [69]:
pd.DataFrame(frames_metadata).to_csv("data/frames_metadata.csv", index=False)

In [66]:
failed_frames

[]

### Discussion
<blockquote>So I have used a combination of the proposed methods after all.</blockquote>
<blockquote>I have used the <code>scenedetect</code> to select the key-frames within a video clip, then I filled in with the in-between frames to reach the required number of frames. All of the selected frames were carefully saved under directories with the same name as the video. One can find them in the "data/SelectedFrames" directory. </blockquote>
<blockquote>I have also created a csv file containing metadata regarding the video clips and their frames. The csv file, <code>frames_metadata</code> has a VideoID column to uniquely identify the video,and regarding the frames, Key_Frames, and Total_Frames columns which represent the number of key-frames and the total number of frames.</blockquote>
<blockquote>There are no failed frame extraction and selection.</blockquote>
<blockquote>The next step in this pipeline is to resize the extracted frames, and normalize pixel values for the CNN.</blockquote>

## Feature Extraction

In [38]:
def frame_preprocess(frame_path):
    img = load_img(frame_path, target_size=(224, 224)) 
    img = img_to_array(img) 
    img = preprocess_input(img)
    
    return img

In [31]:
from keras.applications.vgg16 import VGG16
from keras.models import Model

def load_model():
    model = VGG16(weights="imagenet", include_top=True, input_shape=(224, 224, 3))
    out = model.layers[-2].output
    model_final = Model(inputs=model.input, outputs=out)
    return model_final

In [46]:
def extract_features(in_dir_path, out_dir_path, model):
    for video_name in tqdm(os.listdir(in_dir_path), desc="Extracting features"):
        video_path = os.path.join(in_dir_path, video_name)
        
        frames = []
        frame_files = [
            os.path.join(video_path, fname)
            for fname in os.listdir(video_path)
            if fname.lower().endswith(".jpg")
        ]

        for frame_path in frame_files:
            frame = frame_preprocess(frame_path)
            frames.append(frame)

        video_input = np.array(frames) 

        features = model.predict(video_input, batch_size=32)
        features = np.array(features) 

        video_out_path = os.path.join(out_dir_path, video_name)
        os.makedirs(video_out_path, exist_ok=True)
        np.save(os.path.join(video_out_path, f"{video_name}.npy"), features)

In [47]:
model = load_model()
extract_features('data/SelectedFrames', 'data/ExtractedFeatures', model)

Extracting features:   0%|                                     | 0/1970 [00:00<?, ?it/s]

1/2 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/stepWARNING:tensorflow:5 out of the last 19 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x7256e829ca40> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 3s/step


Extracting features:   0%|                           | 1/1970 [00:07<4:14:58,  7.77s/it]

2/2 ━━━━━━━━━━━━━━━━━━━━ 8s 3s/step


Extracting features:   0%|                           | 2/1970 [00:15<4:18:48,  7.89s/it]

...

Extracting features: 100%|███████████████████████▉| 1968/1970 [1:35:34<00:05,  2.73s/it]

2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 937ms/step


Extracting features: 100%|███████████████████████▉| 1969/1970 [1:35:36<00:02,  2.72s/it]

2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 924ms/step


Extracting features: 100%|████████████████████████| 1970/1970 [1:35:39<00:00,  2.91s/it]
